## Evaluating the models with the Test Set 

In [1]:
import sys 
sys.path.append('../')
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

import time
import timeit
from NN import activation_functions, loss_functions
import NN.network as network

from utils import dataset_helper
from utils import custom_scores
from importlib import reload 


base_dir = '../data/fashion'
print(os.listdir(base_dir))

['train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz']


/home/azael/Trabalhos/MO444/MO444/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from utils import mnist_reader, dataset_helper
X, y = mnist_reader.load_mnist('../data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('../data/fashion', kind='t10k')

In [3]:
X = X.copy() / 255.

Given the dtypes, there is no possibility of negative values in the dataset. 

In [4]:
nclasses =10
X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.05, random_state=42)    
y_train = dataset_helper.one_hot_encode(y_train, nclasses)
y_val = dataset_helper.one_hot_encode(y_val, nclasses)

((57000, 784), (3000, 784), (57000, 10), (3000, 10))

In [5]:
lr = 0.0001
b_sz = 256
eps = np.finfo(np.float64).eps
nfeatures  = X.shape[1]
epoch_sz = X.shape[0]
max_iter = 200 * (epoch_sz // b_sz) 
print_interval = 10 * (epoch_sz // b_sz) 
decay_iteractions= 300 * (epoch_sz // b_sz) 
decay_rate = 0.5

In [9]:
import time
import timeit
from NN import activation_functions, loss_functions
import NN.network as network

from utils import dataset_helper
from utils import custom_scores
from importlib import reload 
reload(custom_scores)
reload(dataset_helper)
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)


h1 = network.Layer(nfeatures, 512, 'relu',  label="H1")
o1 = network.Layer(512, nclasses, 'softmax', label="soft")

model = network.NN(loss='cross_entropy')
model.add_layer(h1)
model.add_layer(o1)
model.summary()

print("")
model.fit(X_train, y_train, max_iter=max_iter, 
          lr=lr, epsilon=eps, b_sz = b_sz,   
          X_val=X_val, Y_val=y_val,
          print_interval=print_interval)
iteraction_log = network.get_iteration_log()

Y_ = np.array(model.predict(X_test)).argmax(axis=-1)
reload(custom_scores)
custom_scores.evaluate_multiclass(y_test=y_test.argmax(axis=-1), y_pred=Y_)

Model Summary
-------------------------------
H1      (input=784, neurons=512, activation=relu)
soft    (input=512, neurons=10, activation=softmax)
-------------------------------

Shuffled
It: 2340 Batch: 110 Epoch 10 Train Loss: 0.07748090 lr: 0.000100 Val Loss: 0.06985190 Val Acc 0.85833333
It: 4680 Batch: 220 Epoch 20 Train Loss: 0.06029129 lr: 0.000100 Val Loss: 0.07058425 Val Acc 0.85666667
It: 7020 Batch: 107 Epoch 31 Train Loss: 0.05452196 lr: 0.000100 Val Loss: 0.06000391 Val Acc 0.88233333
It: 9360 Batch: 217 Epoch 41 Train Loss: 0.05074901 lr: 0.000100 Val Loss: 0.06125652 Val Acc 0.87600000
It: 11700 Batch: 104 Epoch 52 Train Loss: 0.04772342 lr: 0.000100 Val Loss: 0.05715281 Val Acc 0.88966667
It: 14040 Batch: 214 Epoch 62 Train Loss: 0.04527599 lr: 0.000100 Val Loss: 0.05525437 Val Acc 0.89300000
It: 16380 Batch: 101 Epoch 73 Train Loss: 0.04308628 lr: 0.000100 Val Loss: 0.05378358 Val Acc 0.89633333
It: 18720 Batch: 211 Epoch 83 Train Loss: 0.04112578 lr: 0.000100 Val Lo

KeyboardInterrupt: 

In [ ]:
reload(custom_scores)
reload(dataset_helper)
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)


h1 = network.Layer(nfeatures, 512, 'relu',  label="H1")
o1 = network.Layer(512, nclasses, 'softmax', label="soft")

model = network.NN(loss='cross_entropy')
model.add_layer(h1)
model.add_layer(h2)
model.add_layer(h3)
model.add_layer(o1)
model.summary()

print("")
model.fit(X_train, y_train, max_iter=max_iter, 
          lr=lr, epsilon=eps, b_sz = b_sz,
          X_val=X_val, Y_val=y_val,
          print_interval=print_interval)
iteraction_log = network.get_iteration_log()

Y_ = np.array(model.predict(X_val)).argmax(axis=-1)
reload(custom_scores)
custom_scores.evaluate_multiclass(y_val=y_val.argmax(axis=-1), y_pred=Y_)
iteraction_log.index = iteraction_log.it
iteraction_log.error_train.plot()
iteraction_log.error_val.plot()